In [72]:
import requests
import time
import uuid
import json

In [59]:
def parse_from_url(url: str):
    coords = []
    url = url.rsplit('/', maxsplit=1)[1]
    for coord in url.split(','):
        x, y = coord.split('%20')
        y = float(y)
        x = float(x)
        coords.append((x, y))
    return coords

def find_deltas(url: str):
    coords = parse_from_url(url)
    delta_x = max([x[0] for x in coords]) - min([x[0] for x in coords])
    delta_y = max([y[1] for y in coords]) - min([y[1] for y in coords])
    return delta_x, delta_y

def build_url(q4, q1, q2, q3):
    base = 'https://www.mlslistings.com/api/geo/intersect/'
    base += f"{q4[1]}%20{q4[0]},"
    base += f"{q1[1]}%20{q1[0]},"
    base += f"{q2[1]}%20{q2[0]},"
    base += f"{q3[1]}%20{q3[0]},"
    base += f"{q4[1]}%20{q4[0]}"
    return base


In [103]:
coords = parse_from_url('https://www.mlslistings.com/api/geo/intersect/-122.0706467871094%2037.433939536790675,-122.05725719970705%2037.18435313247095,-121.72320416015627%2037.182711973516525,-121.72663738769533%2037.442390188082555,-122.0706467871094%2037.433939536790675')
print(coords)
xmin = min([x[0] for x in coords])
xmax = max([x[0] for x in coords])
ymin = min([y[1] for y in coords])
ymax = max([y[1] for y in coords])

delta_x, delta_y = find_deltas('https://www.mlslistings.com/api/geo/intersect/-121.91185603598883%2037.2884878579338,-121.863619189065%2037.289034147836475,-121.86576495627692%2037.32740108598408,-121.91099772910407%2037.32801534386237,-121.91185603598883%2037.2884878579338')
print(delta_x)
print(delta_y)
#delta_x = delta_x/10.0
#print(delta_x)
delta_x = 0.035
delta_y = 0.035

[(-122.0706467871094, 37.433939536790675), (-122.05725719970705, 37.18435313247095), (-121.72320416015627, 37.182711973516525), (-121.72663738769533, 37.442390188082555), (-122.0706467871094, 37.433939536790675)]
0.048236846923828125
0.039527485928566364


In [107]:
coord_pairs = []
y = ymin
while y < ymax:
    x = xmax
    #print(f"{x} -> {x+delta_x} -> {xmax}")
    #input('press enter')
    while x > xmin:
        q1 = (y, x-delta_x)
        q2 = (y-delta_y, x-delta_x)
        q3 = (y-delta_y, x)
        q4 = (y, x)
        coord_pairs.append((q4, q1, q2, q3))
        x -= delta_x
    y += delta_y

coord_pairs

[((37.182711973516525, -121.72320416015627),
  (37.182711973516525, -121.75820416015627),
  (37.14771197351653, -121.75820416015627),
  (37.14771197351653, -121.72320416015627)),
 ((37.182711973516525, -121.75820416015627),
  (37.182711973516525, -121.79320416015626),
  (37.14771197351653, -121.79320416015626),
  (37.14771197351653, -121.75820416015627)),
 ((37.182711973516525, -121.79320416015626),
  (37.182711973516525, -121.82820416015626),
  (37.14771197351653, -121.82820416015626),
  (37.14771197351653, -121.79320416015626)),
 ((37.182711973516525, -121.82820416015626),
  (37.182711973516525, -121.86320416015626),
  (37.14771197351653, -121.86320416015626),
  (37.14771197351653, -121.82820416015626)),
 ((37.182711973516525, -121.86320416015626),
  (37.182711973516525, -121.89820416015625),
  (37.14771197351653, -121.89820416015625),
  (37.14771197351653, -121.86320416015626)),
 ((37.182711973516525, -121.89820416015625),
  (37.182711973516525, -121.93320416015625),
  (37.147711973

In [73]:
urls = []
data = {}
for i in coord_pairs:
    urls.append(build_url(*i))

In [77]:
for url in urls:
    if url in data:
        continue
    try:
        print(url)
        r = requests.get(url)
        print(r)
        response_data = r.json()
        response_data['url'] = url
        with open(f'{uuid.uuid4()}.json', 'w') as fp:
            json.dump(response_data, fp)
        data[url] = r.json()
        time.sleep(30)
    except KeyboardInterrupt:
        break

https://www.mlslistings.com/api/geo/intersect/-121.9320942476066%2037.38036955943892,-121.98033109453043%2037.38036955943892,-121.98033109453043%2037.419897045367485,-121.9320942476066%2037.419897045367485,-121.9320942476066%2037.38036955943892
<Response [200]>
https://www.mlslistings.com/api/geo/intersect/-121.98033109453043%2037.38036955943892,-122.02856794145426%2037.38036955943892,-122.02856794145426%2037.419897045367485,-121.98033109453043%2037.419897045367485,-121.98033109453043%2037.38036955943892
<Response [200]>
https://www.mlslistings.com/api/geo/intersect/-122.02856794145426%2037.38036955943892,-122.07680478837808%2037.38036955943892,-122.07680478837808%2037.419897045367485,-122.02856794145426%2037.419897045367485,-122.02856794145426%2037.38036955943892
<Response [200]>
https://www.mlslistings.com/api/geo/intersect/-121.73914685991129%2037.419897045367485,-121.78738370683512%2037.419897045367485,-121.78738370683512%2037.45942453129605,-121.73914685991129%2037.45942453129605,

In [84]:
print(len(data), len(coord_pairs))
from pprint import pprint
pprint(parse_from_url('https://www.mlslistings.com/api/geo/intersect/-121.91185603598883%2037.2884878579338,-121.863619189065%2037.289034147836475,-121.86576495627692%2037.32740108598408,-121.91099772910407%2037.32801534386237,-121.91185603598883%2037.2884878579338'))
print(find_deltas('https://www.mlslistings.com/api/geo/intersect/-121.91185603598883%2037.2884878579338,-121.863619189065%2037.289034147836475,-121.86576495627692%2037.32740108598408,-121.91099772910407%2037.32801534386237,-121.91185603598883%2037.2884878579338'))

56 56
[(-121.91185603598883, 37.2884878579338),
 (-121.863619189065, 37.289034147836475),
 (-121.86576495627692, 37.32740108598408),
 (-121.91099772910407, 37.32801534386237),
 (-121.91185603598883, 37.2884878579338)]
(0.048236846923828125, 0.039527485928566364)
